In [2]:

%load_ext autoreload
%autoreload 2
import pickle
from datetime import datetime
import numpy as np
from qutip import *
import os
from matplotlib import pyplot as plt
import sys
from scipy.integrate import solve_ivp
import sandbox as sb
import importlib
importlib.reload(sb)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<module 'sandbox' from '/home/saha/Desktop/tesis/crimson/polaron_detuning/sandbox.py'>

In [ ]:
N = 10
cavity_diss_rate = (4.3)*2*np.pi*10**6
eff_coupling = (1)*2*np.pi*10**6
input_power = (0)
rabi_freq = (20)*2*np.pi*10**6
qubit_targetz = (0)*2*np.pi*10**6
final_time__mus = 2
time_steps = 1000
tlist = np.linspace(0, final_time__mus, time_steps)
norm = eff_coupling
target_state = 'down'

parameters = {
    "N": N,
    "final_time__mus": final_time__mus,
    "time_steps": time_steps,
    "input_power": input_power,
    "rabi_freq": rabi_freq,
    "eff_coupling": eff_coupling,
    "qubit_targetz": qubit_targetz,
    "cavity_diss_rate": cavity_diss_rate,
    "norm": norm,
    "target_state": target_state
}


parameters = sb.calculate_cavity_drive_amplitude_normal_shift(parameters)
cavity_drive_amplitude = parameters['cavity_drive_amplitude']
parameters = sb.get_initial_cavity_state_normal_shift(parameters)
parameters = sb.get_initial_qubit_state(parameters)

parameters_polaron = sb.calculate_parameters_polaron(parameters)
dynamics_polaron = sb.calculate_full_dynamics(parameters_polaron)

parameters_normal = sb.calculate_parameters_normal_shift(parameters)
dynamics_normal = sb.calculate_full_dynamics(parameters_normal)
dynamics_redfield = sb.calculate_tdep_redfield_dynamics_polaron(parameters_normal)
dynamics_tdep_redfield = sb.calculate_tdep_redfield_dynamics(parameters_normal)


# Calculate expectations values.
d = tensor(destroy(N), qeye(2))   
sz = tensor(qeye(N), sigmaz())  
sx = tensor(qeye(N), sigmax())     
sm = tensor(qeye(N), sigmam()) 

x_exp = expect(sx, dynamics)
z_exp = expect(sz, dynamics)
photon_number = expect(d.dag()*d, dynamics)

x_exp_redfield = expect(sigmax(), dynamics_redfield)
z_exp_redfield = expect(sigmaz(), dynamics_redfield)

x_exp_tredfield = expect(sigmax(), dynamics_tdep_redfield)
z_exp_tredfield = expect(sigmaz(), dynamics_tdep_redfield)

x_exp_tredfield_polaron = expect(sigmax(), dynamics_tdep_redfield_polaron)
z_exp_tredfield_polaron = expect(sigmaz(), dynamics_tdep_redfield_polaron)

# Generate plot
max_time = None
if max_time is None:
    max_time = max(tlist)

time_mask = tlist <= max_time
tlist_plot = tlist[time_mask]

# Create plot
fig, axes = plt.subplots(3, 1, figsize=(6, 9))

# Plot only up to max_time
axes[0].plot(tlist_plot, x_exp[time_mask], color='#008000', label='Full dynamics')
axes[1].plot(tlist_plot, z_exp[time_mask], color='#008000', label='Full dynamics')
axes[2].plot(tlist_plot, photon_number[time_mask], color='#008000', label='Full dynamics')

#Redfield
axes[0].plot(tlist_plot, x_exp_redfield[time_mask], color='#000680', label='Redfield')
axes[1].plot(tlist_plot, z_exp_redfield[time_mask], color="#000680", label='Redfield')

#Tdep-Redfield
axes[0].plot(tlist_plot, x_exp_tredfield[time_mask], color="#811606", label='Tdep Redfield')
axes[1].plot(tlist_plot, z_exp_tredfield[time_mask], color="#811606", label='Tdep Redfield')

#Tdep-Redfield  polaron
axes[0].plot(tlist_plot, x_exp_tredfield_polaron[time_mask], color="#16BDC6", label='Tdep Redfield Polaron')
axes[1].plot(tlist_plot, z_exp_tredfield_polaron[time_mask], color="#16BDC6", label='Tdep Redfield Polaron')

# Format σ_x plot
axes[0].set_ylabel(r'$<\sigma_x>$')
axes[0].legend()
axes[0].set_ylim(-1.1, 1.1)
axes[0].grid(True, alpha=0.3)

# Format σ_z plot  
axes[1].set_ylabel(r'$<\sigma_z>$')
axes[1].legend()
axes[1].set_ylim(-1.1, 1.1)
axes[1].grid(True, alpha=0.3)

# Format photon number plot
axes[2].set_ylabel(r'$<a^\dagger a>$')
axes[2].set_xlabel('Time (μs)')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

params_text = []
params_text.append(r"$\Omega_R$:"+f"{rabi_freq/(2*np.pi*1e6):.1f}")
params_text.append(f"κ: {cavity_diss_rate/(2*np.pi*1e6):.1f}")
params_text.append(r"$\chi$" + f": {eff_coupling/(2*np.pi*1e6):.1f}")
params_text.append(r"$\Omega_z$:"+f"{qubit_targetz/(2*np.pi*1e6):.1f}")
params_text.append(r"$\epsilon_d$:"+f"{cavity_drive_amplitude/(2*np.pi*1e6):.1f}")
params_text.append(r"units (2$\pi$Mhz)")
plt.suptitle(' | '.join(params_text), y=0.98)
plt.tight_layout()
plt.show()